# Implementation using CVX
This is a code to solve the static traffic assignment problem using CVX

In [1]:
import cvxpy
import numpy as np

### First, we load the network from files

In [3]:
network = 'data/braess'
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [11]:
## Please run the annex functions at the end of the code before running these cell
graph, demand = cleaning_input(graph, demand)
nb_links = graph.shape[0]

### We compute the incidence matrix and the matrix which gives the correspondance between path and o,d

In [9]:
delta, route2od = delta_matrix(graph, demand)

In [12]:
u = [10000000 for _ in range(nb_links)]

### We define the CVX variables and the objective function

In [43]:
f_cvx = cvxpy.Variable(nb_links)
h_cvx = cvxpy.Variable((delta.shape[0]))
obj_cvx = np.sum(graph[:,3]*f_cvx + 1/2*graph[:,4]*(f_cvx**2) + 1/3*graph[:,5]*(f_cvx**3) + 1/4*graph[:,6]*(f_cvx**4) + 1/5*graph[:,7]*(f_cvx**5))

In [44]:
constr = [
    f_cvx >= delta.T @ h_cvx,
    h_cvx >= 0,
    # f_cvx <= u,
    # the following constraint correspond to a one o-d demand
    sum(h_cvx) >= demand[0][2]
    ]

    # if there are more than one o-d demand, we should work with route2od
"""
    B = np.zeros(route2od.shape[0], np.max(route2od[:,0])+1)
    for i in range(B.shape[1]):
        B[:,i] = 1 * (int(route2od[i])==int(demand[i][0]))
    # add the constraints
    B @ h_cvx == demand[:,2]
"""
print()

### Now, we solve the problem
And we verify the solution (i.e. the Wardrop equilibrium condition)

In [45]:
pb = cvxpy.Problem(cvxpy.Minimize(obj_cvx), constr)
pb.solve()

inf

In [46]:
h_sol = h_cvx.value
f_sol = f_cvx.value

Here we observe that cvx cannot solve the STA because the values of the objective function are to big. :-/ 

In [47]:
print(delta.T @ h_sol)
print((route2od+1) @ h_sol )
print(demand[0][2])
print(f_cvx.value)

TypeError: Object arrays are not currently supported

## Annex functions; to clean the data and compute the paths from the links

In [4]:
def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

In [5]:
graph, demand = cleaning_input(graph, demand)
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

In [8]:
from collections import defaultdict 
   
# This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
        
        if u ==d: 
            path_tmp = np.zeros(shape=(self.V))
            for i in range(len(path)-1):
                path_tmp[graph_dict[path[i]][path[i+1]]] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, self.V))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, self.V)), axis=0)
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path) 
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop()
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d,visited, path) 
        return self.paths_m 

# argument graph
def delta_matrix(graph, demand):
    route2od = np.array([])
    paths_matrix = np.array([])
    g = Graph(graph.shape[0]) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od